In [1]:
import pandas as pd
from pandarallel import pandarallel
from typing import defaultdict

In [2]:
data_path = '../data/'

### ИПЦ

In [3]:
ipc = pd.read_feather(data_path + '/external/ipc.feather')

In [4]:
# ipc.columns = ipc.columns.to_flat_index()
# ipc.reset_index(inplace=True)
# ipc.rename(columns={'level_0': 'area', 'level_1': 'subject'}, inplace=True)
# months_mapper = {
#     'январь': 1,
#     'февраль': 2,
#     'март': 3,
#     'апрель': 4,
#     'май': 5,
#     'июнь': 6,
#     'июль': 7,
#     'август': 8,
#     'сентябрь': 9,
#     'октябрь': 10,
#     'ноябрь': 11,
#     'декабрь': 12
# }
# ipc.columns = ['area', 'subject'] + [pd.to_datetime(str(col[0]) + '-' + str(months_mapper[col[1]]), errors='coerce').date()
#                                      for col in ipc.iloc[:, 2:].columns]

In [5]:
# ipc = pd.melt(ipc, id_vars=ipc.columns[:2], value_vars=ipc.columns[2:], var_name='month', value_name='ipc')

In [6]:
# print(ipc.shape)

In [7]:
# ipc.area.nunique()

In [8]:
def columns_report(data: pd.DataFrame) -> None:

    print('Rows:', data.shape[0], end='\n'*2)

    stats = defaultdict(list)
    for col in data.columns:
        stats['Column'].append(col)
        stats['Unique'].append(data[col].nunique())
        stats['Duplicates'].append(data[col].duplicated().sum())
        stats['Missing'].append(data[col].isna().sum())
        stats['Missing%'].append(data[col].isna().mean() * 100)
        stats['HitRate%'].append((1 - data[col].isna().mean()) * 100)
    stats = pd.DataFrame(stats)

    print(stats.to_string(index=False))

In [9]:
columns_report(ipc)

Rows: 10170

Column  Unique  Duplicates  Missing  Missing%  HitRate%
  area      11       10159        0       0.0     100.0
  type      11       10159        0       0.0     100.0
   ipc     681        9489        0       0.0     100.0
  date      94       10076        0       0.0     100.0


In [10]:
areas = ['край', 'область', 'республика', 'округ', 'российская федерация', 'москва', 'санкт-петербург']
stop_words = areas[:4]

In [11]:
def rm_stop_words(s: str, stop_words=stop_words):
    return ' '.join(w for w in s.split() if w not in stop_words)

def prep_fedstat_geo(data: pd.DataFrame, subs_to_maintain: list,
                     geo_col: str = 'area') -> pd.DataFrame:

    pandarallel.initialize(progress_bar=False, nb_workers=12)

    data[geo_col] = data[geo_col]\
        .str.strip()\
        .str.lower()\
        .str.replace(r'\s*\(.*\)', '', regex=True)\
        .str.replace(r' - .*', '', regex=True)
    data = data.loc[
        data[geo_col].str.contains('|'.join(subs_to_maintain))].reset_index(drop=True)
    data[geo_col] = data[geo_col].parallel_apply(
        rm_stop_words)\
        .str.strip()

    return data

In [12]:
ipc = prep_fedstat_geo(ipc, areas)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [13]:
ipc.loc[ipc.area == 'город москва столица российской федерации город федерального значения', 'area'] = 'москва'
ipc.loc[ipc.area == 'город санкт-петербург город федерального значения', 'area'] = 'санкт-петербург'

In [14]:
ipc.area.unique()

array(['бурятия', 'забайкальский', 'саха', 'камчатский', 'приморский',
       'хабаровский', 'амурская', 'магаданская', 'сахалинская',
       'еврейская автономная', 'чукотский автономный'], dtype=object)

In [15]:
ipc['date'] = pd.to_datetime(ipc.date, yearfirst=True)

In [16]:
ipc = pd.pivot(ipc, index=['area', 'date'], columns=['type'], values=['ipc']).reset_index().droplevel(0, axis=1)
ipc.columns = ['area', 'date'] + ipc.columns[2:].tolist()
ipc.sort_values(['area', 'date'], ascending=[True, True], inplace=True)

In [17]:
columns_report(ipc)

Rows: 1034

                                                                Column  Unique  Duplicates  Missing  Missing%   HitRate%
                                                                  area      11        1023        0  0.000000 100.000000
                                                                  date      94         940        0  0.000000 100.000000
                                    Базовый индекс потребительских цен     193         841        0  0.000000 100.000000
                                                   Все товары и услуги     214         820        0  0.000000 100.000000
                                                           Газ сетевой      24        1009      742 71.760155  28.239845
                                           Жилищно-коммунальные услуги     176         857      242 23.404255  76.595745
                      Жилищные услуги без аренды квартир у частных лиц     163         871        0  0.000000 100.000000
                    

In [18]:
ipc.drop(columns=['Газ сетевой'], inplace=True)

In [19]:
ipc.to_feather(data_path + 'external/ipc_preprocessed.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Ставка ЦБ

In [35]:
cb_rate = pd.read_feather(data_path + 'external/rate.feather')

In [36]:
cb_rate.rename(columns={
    'Ключевая ставка, % годовых': 'cb_rate',
    'Дата': 'date'
}, inplace=True)

In [37]:
cb_rate['cb_rate'] = cb_rate.cb_rate / 100

In [39]:
cb_rate.to_feather(data_path + 'external/cb_rate_preprocessed.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
